In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
from bs4 import BeautifulSoup, Comment
import time

url = "https://www.baseball-reference.com/players/l/lindofr01.shtml"
CHROMEDRIVER_PATH = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\chromedriver-win64\chromedriver.exe"
OUTPUT_PATH = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\MLB_2018_2025_with_Salaries.csv"
hitting_data = pd.read_csv("C:\\Users\\Isaac\\OneDrive\\Documents\\fall 2025 semester\\STAT 386\\MLB_2018_2025_Cleaned.csv")

URL_COLUMN = "Player_Link"
YEAR_COLUMN  = "Year"

hitting_data["Salary"] = 0                # integer version
hitting_data["Salary_formatted"] = ""

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# Print driver info to confirm version
print("Driver started successfully!")

driver.get(url)
time.sleep(6)

page_source = driver.page_source
driver.quit()

soup = BeautifulSoup(page_source, "html.parser")
print("'br-salaries' in page?", "br-salaries" in page_source)

comments = soup.find_all(string=lambda text: isinstance(text, Comment))

for comment in comments:
    if "br-salaries" in comment:
        comment_soup = BeautifulSoup(comment, "html.parser")
        table = comment_soup.find("table", id="br-salaries")
        if table:
            print("Lindor Salaries (2016–2031 + future years):")
            print("Year  Age  Team                     Salary           Service Time")
            print("-" * 75)
           
            for row in table.find("tbody").find_all("tr"):
                if row.get("class") and ("spacer" in row.get("class") or "status_head" in row.get("class")):
                    continue
                cells = row.find_all(["th", "td"])
                if len(cells) < 4:
                    continue
                year   = cells[0].get_text(strip=True)
                age    = cells[1].get_text(strip=True)
                team   = cells[2].get_text(strip=True).replace("\xa0", " ")
                salary = cells[3].get_text(strip=True)
                srv    = cells[4].get_text(strip=True) if len(cells) > 4 else ""
                if cells[3].get("data-future"):
                    salary += "  ← future"
                print(f"{year:4}  {age:3}  {team:23}  {salary:18}  {srv}")
           
            tfoot = table.find("tfoot")
            if tfoot:
                total = tfoot.find("td", {"data-stat": "salary_total"})
                if total:
                    print(f"\nCareer total to date: {total.get_text(strip=True)}")
            break
else:
    print("Salaries table not found in any comment")


Driver started successfully!
'br-salaries' in page? True
Lindor Salaries (2016–2031 + future years):
Year  Age  Team                     Salary           Service Time
---------------------------------------------------------------------------
2016  22   Cleveland Indians        $540,300            0.113
2017  23   Cleveland Indians        $579,300            1.113
2018  24   Cleveland Indians        $623,200            2.113
2019  25   Cleveland Indians        $10,850,000         3.113
2020  26   Cleveland Indians        $17,500,000         4.113
2021  27   New York Mets            $22,300,000         5.113
2022  28   New York Mets            $34,100,000         6.113
2023  29   New York Mets            $34,100,000         7.113
2024  30   New York Mets            $34,100,000         8.113
2025  31   New York Mets            $34,100,000         9.113
2026  32   New York Mets            $34,100,000  ← future  10.113
2027  33   New York Mets            $34,100,000  ← future  
2028  34   

In [3]:
# FINAL_REAL_ONE_that_works_dec2025.py
import pandas as pd
import time
import random
import os
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup, Comment

# Prevent computer from sleeping (Windows)
os.system('powercfg -change -standby-timeout-ac 0')
os.system('powercfg -change -standby-timeout-dc 0')

INPUT  = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\MLB_2018_2025_Cleaned.csv"
OUTPUT = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\MLB_Salaries_2018_2025_FINAL.csv"

# -------------------------------------------------
def get_salaries(html):
    soup = BeautifulSoup(html, "html.parser")
    for comment in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if "br-salaries" in comment:
            cs = BeautifulSoup(comment, "html.parser")
            table = cs.find("table", id="br-salaries")
            if table:
                sal = {}
                for row in table.find_all("tr")[1:]:
                    c = row.find_all("td")
                    if len(c) >= 4 and c[0].get_text(strip=True).isdigit():
                        y = int(c[0].get_text(strip=True))
                        if 2018 <= y <= 2025:
                            s = c[3].get_text(strip=True).replace("$","").replace(",","")
                            sal[y] = int(s) if s else 0
                return sal
    return {}

# -------------------------------------------------
df = pd.read_csv(INPUT)
df = df[df["Year"].between(2018,2025)][["Player","Year","Player_Link"]].reset_index(drop=True)
df["Player_Link"] = "https://www.baseball-reference.com" + df["Player_Link"].astype(str)

cache = {}
total = df["Player_Link"].nunique()
print(f"Starting {total} unique players with Playwright (invisible, Cloudflare-proof)")

with sync_playwright() as p:
    browser = p.chromium.launch(headless=True)          # truly invisible
    context = browser.new_context(
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        viewport={"width": 1920, "height": 1080},
        java_script_enabled=True
    )
    # This single line is why it never gets Cloudflare blocked
    context.add_init_script("Object.defineProperty(navigator, 'webdriver', {get: () => false})")
    
    page = context.new_page()

    for i, link in enumerate(df["Player_Link"].unique(), 1):
        name = df[df["Player_Link"] == link]["Player"].iloc[0]
        if i <= 5 or i % 50 == 0:
            print(f"{i:4d}/{total} → {name}")

        try:
            page.goto(link, wait_until="domcontentloaded", timeout=30000)
            page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(1.8)
            salaries = get_salaries(page.content())
            for n in df[df["Player_Link"]==link]["Player"].unique():
                cache[n] = salaries
        except Exception as e:
            print(f"   failed {name} → will be 0")
        
        time.sleep(random.uniform(0.9, 2.1))   # 30–50 players per minute

    browser.close()

# restore normal sleep
os.system('powercfg -change -standby-timeout-ac 15')
os.system('powercfg -change -standby-timeout-dc 5')

df["Salary"] = df.apply(lambda r: cache.get(r["Player"], {}).get(r["Year"], 0), axis=1)
df["Salary_Formatted"] = df["Salary"].apply(lambda x: f"${x:,}" if x > 0 else "")
df[["Player","Year","Salary_Formatted"]].to_csv(OUTPUT, index=False)

print("\n100 % DONE – NO MORE CLOUDFLARE PROBLEMS EVER AGAIN")
input("Press Enter to close...")

Starting 1027 unique players with Playwright (invisible, Cloudflare-proof)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup, Comment
import time
import random

# ---------------- CONFIG ----------------
INPUT_CSV  = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\MLB_2018_2025_Cleaned.csv"
OUT_CSV    = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\MLB_Salaries_2018_2025_FINAL.csv"
DRIVER     = r"C:\Users\Isaac\OneDrive\Documents\fall 2025 semester\STAT 386\chromedriver-win64\chromedriver.exe"

# ---------------- DRIVER SETUP ----------------
def make_driver():
    opts = Options()
    opts.add_argument("--headless")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(service=Service(DRIVER), options=opts)
    driver.set_page_load_timeout(25)
    driver.implicitly_wait(10)
    return driver

# ---------------- EXTRACT SALARIES FROM PAGE ----------------
def get_player_salaries_2018_2025(html):
    soup = BeautifulSoup(html, "html.parser")
    salaries = {}

    # Method 1: ALWAYS start with comments (this is where EVERY table lives in 2025)
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        if "all_br-salaries" in str(comment) or "br-salaries" in str(comment):
            comment_soup = BeautifulSoup(comment, "html.parser")
            table = comment_soup.find("table", {"id": "br-salaries"})
            if table:
                for row in table.find_all("tr")[1:]:  # skip header
                    cells = row.find_all("td")
                    if len(cells) >= 4:
                        year_text = cells[0].get_text(strip=True)
                        salary_text = cells[3].get_text(strip=True).replace("$", "").replace(",", "")
                        if year_text.isdigit() and 2018 <= int(year_text) <= 2025:
                            try:
                                salaries[int(year_text)] = int(salary_text) if salary_text else 0
                            except:
                                salaries[int(year_text)] = 0
                break  # found the table
    if salaries:
        return salaries

    # Method 2: Fallback to direct DOM (rare, but covers any JS-injected cases)
    table = soup.find("table", {"id": "br-salaries"})
    if table:
        for row in table.find_all("tr")[1:]:  # skip header
            cells = row.find_all("td")
            if len(cells) >= 4:
                year_text = cells[0].get_text(strip=True)
                salary_text = cells[3].get_text(strip=True).replace("$", "").replace(",", "")
                if year_text.isdigit() and 2018 <= int(year_text) <= 2025:
                    try:
                        salaries[int(year_text)] = int(salary_text) if salary_text else 0
                    except:
                        salaries[int(year_text)] = 0
    return salaries

# ---------------- MAIN ----------------
print("Loading dataset...")
df = pd.read_csv(INPUT_CSV)

# Keep only 2018–2025
df = df[df["Year"].between(2018, 2025)].copy()
df = df[["Player", "Year", "Player_Link"]].reset_index(drop=True)

# Fix links
df["Player_Link"] = df["Player_Link"].apply(
    lambda x: "https://www.baseball-reference.com" + x if str(x).startswith("/") else str(x)
)

print(f"Total rows to enrich: {len(df)}")
print("Starting salary scraping... (this will take ~20–30 minutes)")

# We'll store salaries here: {player_name: {year: salary}}
player_salary_cache = {}
driver = make_driver()

try:
    unique_links = df["Player_Link"].unique()
    print(f"Unique player pages to visit: {len(unique_links)}")

    for i, link in enumerate(unique_links):
        players_using_this_link = df[df["Player_Link"] == link]["Player"].unique()
        player_name = players_using_this_link[0]  # use first as display name

        if (i + 1) % 20 == 0:
            print(f"   {i+1}/{len(unique_links)} → {player_name}")

        # Restart driver every 75 players
        if (i + 1) % 75 == 0:
            driver.quit()
            time.sleep(3)
            driver = make_driver()
            print("   ↻ Driver restarted")

        salaries = {}
        for attempt in range(3):
            try:
                driver.get(link)
                WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")

                # Scroll to bottom to ensure full page source (triggers any lazy-load of comments)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)

                salaries = get_player_salaries_2018_2025(driver.page_source)
                if salaries:  # got data
                    break

            except Exception as e:
                if attempt < 2:
                    print(f"   Retry {attempt+1} for {player_name}: {str(e)[:50]}")
                    time.sleep(10)
                else:
                    print(f"   Failed after 3 tries: {player_name}")

        # Cache results for all players sharing this link
        for name in players_using_this_link:
            player_salary_cache[name] = salaries

        time.sleep(random.uniform(1.2, 2.8))

finally:
    driver.quit()

# ---------------- APPLY SALARIES BACK IN ORIGINAL ORDER ----------------
print("Applying salaries to original dataset...")
df["Salary"] = df.apply(
    lambda row: player_salary_cache.get(row["Player"], {}).get(row["Year"], 0),
    axis=1
)

df["Salary_Formatted"] = df["Salary"].apply(lambda x: f"${x:,}" if x > 0 else "")

# Final output — exactly your original row order!
final = df[["Player", "Year", "Salary_Formatted"]].copy()

final.to_csv(OUT_CSV, index=False)

print("\nDONE! 100% complete.")
print(f"Saved {len(final):,} rows → {OUT_CSV}")
print("\nFirst 10 rows:")
print(final.head(10).to_string(index=False))

print("\nTop 10 highest paid (2018–2025):")
top10 = final.copy()
top10["Salary_Num"] = top10["Salary_Formatted"].str.replace(r"[$,]", "", regex=True).fillna("0").astype(int)
print(top10.nlargest(10, "Salary_Num")[["Player", "Year", "Salary_Formatted"]].to_string(index=False))

Loading dataset...
Total rows to enrich: 3518
Starting salary scraping... (this will take ~20–30 minutes)
Unique player pages to visit: 1027
   Retry 2 for Francisco Lindor: Message: timeout: Timed out receiving message from
   Failed after 3 tries: Whit Merrifield
   Failed after 3 tries: Charlie Blackmon
   20/1027 → Carlos Santana
   Failed after 3 tries: Carlos Santana
   Retry 2 for Khris Davis: Message: timeout: Timed out receiving message from
   Retry 1 for Christian Yelich: Message: timeout: Timed out receiving message from
   40/1027 → Javier BÃ¡ez
   Retry 1 for Michael Brantley: Message: 

   60/1027 → Adam Jones
   Retry 2 for Stephen Piscotty: Message: timeout: Timed out receiving message from
   Failed after 3 tries: OdÃºbel Herrera
   ↻ Driver restarted
   Retry 2 for Brandon Crawford: Message: timeout: Timed out receiving message from
   80/1027 → Eddie Rosario
   Failed after 3 tries: DJ LeMahieu
   Retry 1 for Xander Bogaerts: Message: timeout: Timed out receiving me